# Imports

In [1]:
import re
import numpy as numpy
import pandas as pd
from pprint import pprint 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#Spacy for lemmatization
import spacy

#Plotting tools
import pyLDAvis
import pyLDAvis.gensim #Important
import matplotlib.pyplot as plt 
#%matplotlib inline

#Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


#### Warning for pool and address sizes don't dissappear <-- SpaCy related

# Stopwords

In [2]:
#NLTK stop words
from nltk.corpus import stopwords
stop_words= stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

#### Import Dataset

In [3]:
#Import Dataset
data = pd.read_csv('articles1.csv', error_bad_lines=False);
data_text = data[['content']]
data_text = data_text.reset_index()
documents = data_text
data_text.head()

,index,content
0,0,WASHINGTON — Congressional Republicans have...
1,1,"After the bullet shells get counted, the blood..."
2,2,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,"Death may be the great equalizer, but it isn’t..."
4,4,"SEOUL, South Korea — North Korea’s leader, ..."


#### Clean emails

In [4]:
#Convert to List
data = data_text.content.values.tolist()

#Remove Emails
data = [re.sub('\S*@\S*\s?', '',sent) for sent in data]
#Remove new line characters
data = [re.sub('\s+',' ',sent) for sent in data]
#Remove distracting single qquotes
data = [re.sub("\'", "", sent) for sent in data]
#Print data from email after cleaning up with REGEX
pprint(data[:1])

['WASHINGTON — Congressional Republicans have a new fear when it comes to '
 'their health care lawsuit against the Obama administration: They might win. '
 'The incoming Trump administration could choose to no longer defend the '
 'executive branch against the suit, which challenges the administration’s '
 'authority to spend billions of dollars on health insurance subsidies for and '
 'Americans, handing House Republicans a big victory on issues. But a sudden '
 'loss of the disputed subsidies could conceivably cause the health care '
 'program to implode, leaving millions of people without access to health '
 'insurance before Republicans have prepared a replacement. That could lead to '
 'chaos in the insurance market and spur a political backlash just as '
 'Republicans gain full control of the government. To stave off that outcome, '
 'Republicans could find themselves in the awkward position of appropriating '
 'huge sums to temporarily prop up the Obama health care law, angerin

#### Break Sentences into Words

In [5]:
#Break down each sentence into a list of words through TOKENIZATION
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) #Removes Punctuations
data_words = list(sent_to_words(data))
#result from breaking down each sentence into a list of words through TOKENIZATION
print(data_words[:1])

[['washington', 'congressional', 'republicans', 'have', 'new', 'fear', 'when', 'it', 'comes', 'to', 'their', 'health', 'care', 'lawsuit', 'against', 'the', 'obama', 'administration', 'they', 'might', 'win', 'the', 'incoming', 'trump', 'administration', 'could', 'choose', 'to', 'no', 'longer', 'defend', 'the', 'executive', 'branch', 'against', 'the', 'suit', 'which', 'challenges', 'the', 'administration', 'authority', 'to', 'spend', 'billions', 'of', 'dollars', 'on', 'health', 'insurance', 'subsidies', 'for', 'and', 'americans', 'handing', 'house', 'republicans', 'big', 'victory', 'on', 'issues', 'but', 'sudden', 'loss', 'of', 'the', 'disputed', 'subsidies', 'could', 'conceivably', 'cause', 'the', 'health', 'care', 'program', 'to', 'implode', 'leaving', 'millions', 'of', 'people', 'without', 'access', 'to', 'health', 'insurance', 'before', 'republicans', 'have', 'prepared', 'replacement', 'that', 'could', 'lead', 'to', 'chaos', 'in', 'the', 'insurance', 'market', 'and', 'spur', 'politic

### Bigram & Trigram Model Building

In [6]:
#Build bigram & Trigram models
bigram = gensim.models.Phrases(data_words,min_count=5,threshold=100) #Higher threshold = fewer phrases
trigram = gensim.models.Phrases(bigram[data_words],threshold=100)
#Faster way to get sentence clubbed as birgram/trigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
#See trigram example
print("-----------------------------")
print(trigram_mod[bigram_mod[data_words[0]]])

-----------------------------
['washington', 'congressional', 'republicans', 'have', 'new', 'fear', 'when', 'it', 'comes', 'to', 'their', 'health_care', 'lawsuit', 'against', 'the', 'obama', 'administration', 'they', 'might', 'win', 'the', 'incoming', 'trump', 'administration', 'could', 'choose', 'to', 'no', 'longer', 'defend', 'the', 'executive_branch', 'against', 'the', 'suit', 'which', 'challenges', 'the', 'administration', 'authority', 'to', 'spend', 'billions', 'of', 'dollars', 'on', 'health_insurance', 'subsidies', 'for', 'and', 'americans', 'handing', 'house', 'republicans', 'big', 'victory', 'on', 'issues', 'but', 'sudden', 'loss', 'of', 'the', 'disputed', 'subsidies', 'could', 'conceivably', 'cause', 'the', 'health_care', 'program', 'to', 'implode', 'leaving', 'millions', 'of', 'people', 'without', 'access', 'to', 'health_insurance', 'before', 'republicans', 'have', 'prepared', 'replacement', 'that', 'could', 'lead', 'to', 'chaos', 'in', 'the', 'insurance', 'market', 'and', 's

#### Functions: Stopwords, Bi/Trigrams, Lemmatization

In [7]:
#define functions for stopwords, bi/trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out= []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#### Call Functions and display

In [8]:
#remove Stop words
data_words_nostops = remove_stopwords(data_words)
#form bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
#finitialize spacy 'en' model
nlp = spacy.load('en', disable=['parser','ner'])
#Do lemmatization keeping only nouns, adjectives, verbs and adverbs
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#Print data after removing stop words, form bigram and lemmatizing data
print("-------------------------------------")
print(data_lemmatized[:1])

c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\python\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


-------------------------------------
[['washington', 'congressional', 'republican', 'new', 'fear', 'come', 'health_care', 'lawsuit', 'obama', 'administration', 'may', 'win', 'incoming', 'trump', 'administration', 'could', 'choose', 'long', 'defend', 'executive_branch', 'suit', 'challenge', 'administration', 'authority', 'spend', 'billion', 'dollar', 'health_insurance', 'subsidy', 'american', 'hand', 'house', 'republican', 'big', 'victory', 'issue', 'sudden', 'loss', 'dispute', 'subsidy', 'could', 'conceivably', 'because', 'health_care', 'program', 'implode', 'leave', 'million', 'people', 'access', 'health_insurance', 'republican', 'prepare', 'replacement', 'could', 'lead', 'chaos', 'insurance', 'market', 'spur', 'political', 'backlash', 'republican', 'gain', 'full', 'control', 'government', 'stave', 'outcome', 'republican', 'could', 'find', 'awkward', 'position', 'appropriating', 'huge_sum', 'temporarily', 'prop', 'obama', 'health_care', 'law', 'anger', 'conservative', 'voter', 'deman

### Create Dictionary and Corpus

In [9]:
#Create Dict
id2word = corpora.Dictionary(data_lemmatized)
#Create Corpus
texts = data_lemmatized
#Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
#View
print(corpus[:1])

#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 13), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 3), (36, 1), (37, 3), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 5), (46, 3), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 8), (57, 2), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 5), (93, 1), (94, 1), (95, 1), (96, 1), (97, 3), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1

## Build LDA Model

In [11]:
# Build LDA model
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,id2word=id2word,
num_topics=20, random_state=100, chunksize=100, passes=10, per_word_topics=True)
print("TEST: DONE LDA MODEL BUILDING")

# Print the Keyword in the 20 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

TEST: DONE LDA MODEL BUILDING


### Perplexity and coherence

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO")


Perplexity:  -8.13239705518372

Coherence Score:  0.4777839945999946
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO


# Visualize

In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

c:\python\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.189874 -0.029147       1        1  12.487844
17    -0.094352  0.128951       2        1  11.735405
5     -0.017140 -0.049862       3        1   7.417658
10    -0.059677  0.091673       4        1   6.709064
9     -0.059564 -0.006558       5        1   6.676547
14     0.087107  0.154000       6        1   6.398856
6      0.094199 -0.030002       7        1   6.026930
12    -0.055246 -0.079320       8        1   5.145339
18    -0.037224  0.097409       9        1   4.988475
7     -0.039030  0.018384      10        1   4.335297
3     -0.010427  0.095167      11        1   4.247260
1     -0.109224 -0.003129      12        1   4.083876
4      0.092926  0.073551      13        1   3.708116
8      0.076861  0.022209      14        1   3.485400
11    -0.054680 -0.149743      15        1   3.222265
13    -0.160729 -0.066832      16        1   2.686286
0      0.162942 -0.070817      17        1   2.174764
19     0.114906 -0.145518      18        1   2.123376
16     0.163576  0.093458      19        1   1.401547
2      0.094650 -0.143874      20        1   0.945699, topic_info=      Category           Freq           Term          Total  loglift  logprob
term                                                                         
259    Default  148762.000000          trump  148762.000000  30.0000  30.0000
225    Default  269825.000000            say  269825.000000  29.0000  29.0000
847    Default   30688.000000         police   30688.000000  28.0000  28.0000
215    Default   35571.000000     republican   35571.000000  27.0000  27.0000
5036   Default   12630.000000         russia   12630.000000  26.0000  26.0000
450    Default   38430.000000        country   38430.000000  25.0000  25.0000
859    Default   60226.000000      president   60226.000000  24.0000  24.0000
1256   Default   23362.000000        company   23362.000000  23.0000  23.0000
1153   Default   29787.000000          woman   29787.000000  22.0000  22.0000
3213   Default   23058.000000           vote   23058.000000  21.0000  21.0000
1725   Default   31299.000000        clinton   31299.000000  20.0000  20.0000
83     Default   22067.000000       election   22067.000000  19.0000  19.0000
5037   Default    9952.000000        russian    9952.000000  18.0000  18.0000
383    Default   33038.000000       campaign   33038.000000  17.0000  17.0000
1584   Default   11744.000000        student   11744.000000  16.0000  16.0000
325    Default   26367.000000         attack   26367.000000  15.0000  15.0000
2320   Default    9703.000000          china    9703.000000  14.0000  14.0000
2354   Default   22256.000000          party   22256.000000  13.0000  13.0000
1577   Default   54955.000000          state   54955.000000  12.0000  12.0000
57     Default   12624.000000          court   12624.000000  11.0000  11.0000
4376   Default   12486.000000           isis   12486.000000  10.0000  10.0000
1636   Default   26178.000000          world   26178.000000   9.0000   9.0000
959    Default   13190.000000         school   13190.000000   8.0000   8.0000
163    Default   26458.000000       official   26458.000000   7.0000   7.0000
13009  Default   16351.000000      breitbart   16351.000000   6.0000   6.0000
107    Default   28063.000000     government   28063.000000   5.0000   5.0000
268    Default   13333.000000          voter   13333.000000   4.0000   4.0000
803    Default   16325.000000        officer   16325.000000   3.0000   3.0000
1381   Default   10362.000000    immigration   10362.000000   2.0000   2.0000
214    Default   38350.000000         report   38350.000000   1.0000   1.0000
...        ...            ...            ...            ...      ...      ...
7177   Topic20     245.861847       shanghai     260.241699   4.6042  -6.4009
38760  Topic20     227.598953          hogan     241.415924   4.6021  -6.4781
24593  Topic20     383.075378     gold_